# Team Members 
<br>
Audrey Pohl | u0497618 | u0497618@utah.edu
<br>
Clarissa Seebhom | u0575630 | u0575630@utah.edu
<br>
Joseph Wirthlin | u0936690 | u0936690@utah.edu
<br>

# Background and Motivation 

Our group collectively enjoys outdoor activities in Salt Lake City. We ski, climb, hike, and camp in the wilderness surrounding Salt Lake City. For this reason, we value the Great Salt Lake for its contribution to the local ecosystem. With increased snowfall this winter, we are curious how much it will impact the level of the Salt Lake. Snowfall isn’t the only factor that can impact the water level. We also will analyze the factors of rainfall, total precipitation, and water usage. 

# Project Objectives

Do rainfall and snowfall have different effects on water levels of the GSL? Does precipitation or human factors have more of an impact on water levels of the GSL?


# Background

# Data

There are four sets of data that we are using for this project, precipitation, snow water equivalent, gross domestic product (GDP), and population growth. The precipitation data for the Salt Lake Metropolitan Area comes from the National Weather Service. Snow water equivalent (SWE) is the amount of water in the Utah snowpack from the US Department of Agriculture. GDP comes from the Federal Reserve Economic Data (FRED) database. FRED is a reputable, government source that posts economic data for the entire country. The population data originally comes from the US Census Bureau but was condensed and simplified by MacroTrends, a website that reliably simplifies large datasets. 

Precipitation - https://www.weather.gov/wrh/climate?wfo=slc <br>
SWE -  https://www.nrcs.usda.gov/Internet/WCIS/AWS_PLOTS/basinCharts/POR/WTEQ/assocHUCut3/state_of_utah.html <br>
GDP - https://fred.stlouisfed.org/series/UTNGSP <br>
Population data - https://www.macrotrends.net/states/utah/population#:~:text=The%20population%20of%20Utah%20in,a%201.53%25%20increase%20from%202018. <br>


# Data Processing

The data to be cleaned and processed:

    - lake level (elevation) [m]
    - snow water equivalent (snowfall) [inches]
    - precipitation (salt lake metropolitan area) [inches]
    - gdp of utah (proxy for commercial water usage) [dollars]
    - population of utah (proxy for individual water usage) [people]

In [ ]:
import datetime
import pandas as pd 
import math as m

In [ ]:
###   IMPORT LAKE LEVEL DATA 
lake_level = pd.read_csv('monthly', sep = '\t', comment = '#') 

#function to convert lake data to datetime
def populate_lake_datetime(data, df):
    for i in range(len(data)-1):
        temp = pd.DataFrame([[datetime.date(int(data['year_nu'][i+1]),int(data['month_nu'][i+1]),20), float(data['mean_va'][i+1])]],columns=['date','lake_level'])
        df = pd.concat([df,temp], ignore_index=True)
    return df

lk_lvl_df = pd.DataFrame(columns=['date','lake_level'])
lk_lvl_df = populate_lake_datetime(lake_level, lk_lvl_df)

In [ ]:
###   IMPORT SNOW WATER EQUIVALENT DATA (inches)
snow_water = pd.read_csv('state_of_utah_snow_water.csv') # snow water equivalent

#find the mean swe per month
october_snow = snow_water.loc[:30].mean()
november_snow = snow_water.loc[31:60].mean()
december_snow = snow_water.loc[61:91].mean()
january_snow = snow_water.loc[92:122].mean()
february_snow = snow_water.loc[124:151].mean()
march_snow = snow_water.loc[152:182].mean()
april_snow = snow_water.loc[183:212].mean()
may_snow = snow_water.loc[213:243].mean()
june_snow = snow_water.loc[244:273].mean()
july_snow = snow_water.loc[274:304].mean()
august_snow = snow_water.loc[305:334].mean()
september_snow = snow_water.loc[335:365].mean()

#function to convert swe data to date time
    #one piece of data per month that is on the 20th
def populate_snow_datetime(data, month, start_year, df):
    for i in range(len(data)-10):
        temp = pd.DataFrame([[datetime.date(start_year+i,month,20), data[i]]],columns=['date','swe'])
        df = pd.concat([df,temp], ignore_index=True)
    return df

#populate into a data frame
swe_df = pd.DataFrame(columns=['date','swe'])

swe_df = populate_snow_datetime(october_snow, 10, 1981, swe_df)
swe_df = populate_snow_datetime(november_snow, 11, 1981, swe_df)
swe_df = populate_snow_datetime(december_snow, 12, 1981, swe_df)
swe_df = populate_snow_datetime(january_snow, 1, 1981, swe_df)
swe_df = populate_snow_datetime(february_snow, 2, 1981, swe_df)
swe_df = populate_snow_datetime(march_snow, 3, 1981, swe_df)
swe_df = populate_snow_datetime(april_snow, 4, 1981, swe_df)
swe_df = populate_snow_datetime(may_snow, 5, 1981, swe_df)
swe_df = populate_snow_datetime(june_snow, 6, 1981, swe_df)
swe_df = populate_snow_datetime(july_snow, 7, 1981, swe_df)
swe_df = populate_snow_datetime(august_snow, 8, 1981, swe_df)
swe_df = populate_snow_datetime(september_snow, 9, 1981, swe_df)

swe_df = swe_df.sort_values(by='date')

In [ ]:
###   IMPORT PRECIPITATION DATA (inches)
precipitation = pd.read_csv('precipitation_data.csv')

#function to convert pecipitation data to datetime
def populate_precip_datetime(data, df):

    for i in range(len(data)):
        for j in range(12):
            month = (str)(j+1)
            temp = pd.DataFrame([[datetime.date(int(data['Year'][i]),int(j+1),20), float(data[month][i])]],columns=['date','precipitation'])
            df = pd.concat([df,temp], ignore_index=True)

    return df

precip_df = pd.DataFrame(columns=['date','precipitation'])
precip_df = populate_precip_datetime(precipitation, precip_df)

In [ ]:
###   IMPORT GDP DATA (some form of dollars)
gdp = pd.read_csv('UTNGSP.csv')

#function to convert gdp data to datetime
def populate_gdp_datetime(data, df):

    for i in range(len(data)):
        year = data['DATE'][i][:4]
        temp = pd.DataFrame([[datetime.date(int(year),int(1),20), float(data['UTNGSP'][i])]],columns=['date','gdp'])
        df = pd.concat([df,temp], ignore_index=True)

    return df

gdp_df = pd.DataFrame(columns=['date','gdp'])
gdp_df = populate_gdp_datetime(gdp, gdp_df)


In [ ]:
###   IMPORT POPULATION GROWTH DATA
pop_growth = pd.read_csv('population_growth.csv')

def populate_pop_datetime(data, df):

    for i in range(len(data)):
        temp = pd.DataFrame([[datetime.date(int(pop_growth['year'][i]),int(1),20), float(data['pop'][i])]],columns=['date','population'])
        df = pd.concat([df,temp], ignore_index=True)

    return df

pop_df = pd.DataFrame(columns=['date','population'])
pop_df = populate_pop_datetime(pop_growth, pop_df)

In [ ]:
###   SLICE ALL DFS TO 1990
swe_monthly = swe_df.loc[swe_df["date"]>datetime.date(1990,1,1)]
precip_monthly = precip_df.loc[precip_df["date"]>datetime.date(1990,1,1)]
lk_lvl_monthly = lk_lvl_df.loc[lk_lvl_df["date"]>datetime.date(1990,1,1)]
gdp_annual = gdp_df.loc[gdp_df["date"]>datetime.date(1990,1,1)]
pop_annual = pop_df.loc[pop_df["date"]>datetime.date(1990,1,1)]

#reset indexes
swe_monthly = swe_monthly.reset_index(drop=True)
precip_monthly = precip_monthly.reset_index(drop=True)
lk_lvl_monthly = lk_lvl_monthly.reset_index(drop=True)
gdp_annual = gdp_annual.reset_index(drop=True)
pop_annual = pop_annual.reset_index(drop=True)

In [ ]:
###    ANNUAL INFORMATION FOR MONTHLY DATA

#function to sum months to get annual data from monthly data
def make_annual_sum(df, str, df_annual):
    curr_year = 1990
    curr_inches = 0
    month = 0

    for i in range(len(df)):
        month += 1
        if(int(df['date'][i].year)==curr_year):
            curr_inches += df[str][i]
        elif(month>12 or i == len(df)):
            temp = pd.DataFrame([[datetime.date(int(curr_year),int(1),20), float(curr_inches)]],columns=['date', str])
            df_annual = pd.concat([df_annual,temp], ignore_index=True)
            curr_year += 1
            curr_inches = 0

    return df_annual

swe_annual = pd.DataFrame(columns=['date','swe'])
swe_annual = make_annual_sum(swe_monthly, 'swe', swe_annual)

precip_annual = pd.DataFrame(columns=['date','precipitation'])
precip_annual = make_annual_sum(precip_monthly, 'precipitation', precip_annual)

def make_annual_lake_level(df, str, df_annual):
    month = 0

    for i in range(len(df)):
        month += 1
        if(int(df['date'][i].month)==1):
            temp = pd.DataFrame([[df['date'][i], float(df[str][i])]],columns=['date', str])
            df_annual = pd.concat([df_annual,temp], ignore_index=True)

    return df_annual

lk_lvl_annual = pd.DataFrame(columns=['date','lake_level'])
lk_lvl_annual = make_annual_lake_level(lk_lvl_monthly, 'lake_level', lk_lvl_annual)

In [ ]:
###    CLEAN DATA

#monthly
swe_monthly 
precip_monthly 
lk_lvl_monthly 

#annual
gdp_annual 
pop_annual
precip_annual
lk_lvl_annual 
swe_annual

# Exploratory Analysis

# Analysis Methodology

# Data Visualization

# Results

# Evaluation

# Ethical Considerations

We must confirm that the data we use will not cause harm to any stakeholders. To prevent harm we identified the primary stakeholders related to this research: Utah residents, the Utah government and policymakers, Utah agriculture, and Utah watershed-reliant businesses. In order to prevent causing harm to individual stakeholders, we decided to use water usage data on a state level so not to unintentionally blame any water users. This is because some historical research left certain stakeholders feeling as if they are receiving a disproportionate amount of the blame and are under increasing pressure to decrease water use as a result of that research. We want to avoid this, so our data will not break down any type of water user to protect those stakeholders from experiencing public scrutiny.


# Project Summary